## Using the Job Submit API

This notebook illustrates the use of the _flux-framework_ Python API to submit jobs in an allocation.

In [ ]:
import json
import os
import re
import flux
from flux.job import JobspecV1

Determine the number of cores and nodes available in your allocation.

In [ ]:
ncores = !flux resource list -no {ncores} --state=up
nc = int(ncores[0])

nnodes = !flux resource list -no {nnodes} --state=up
nn = int(nnodes[0])

`flux.Flux()` creates a new Flux handle which can be used to connect to an interact with a flux broker.

In [ ]:
f = flux.Flux()

The `JobspecV1` class constructs a job request that can be submitted to flux. Here we create a job request that will execute `compute.py` four times 
across two nodes using two cores per execution.

In [ ]:
compute_jobreq = JobspecV1.from_command(
    command=["./flux-workflow-examples/job-submit-api/compute.py", "120"], num_tasks=nn, num_nodes=int(nn/2), cores_per_task=2
)
compute_jobreq.cwd = os.getcwd()
compute_jobreq.environment = dict(os.environ)

The call to `flux.job.submit()` submits the job to the flux broker and returns its _job ID_ once it begins running.

In [ ]:
print(flux.job.submit(f, compute_jobreq))

Create a new job request that executes `io-forwarding.py` once on a single node with one core and submit it as well

In [ ]:
io_jobreq = JobspecV1.from_command(
    command=["./flux-workflow-examples/job-submit-api/io-forwarding.py", "120"], num_tasks=1, num_nodes=1, cores_per_task=1
)
io_jobreq.cwd = os.getcwd()
io_jobreq.environment = dict(os.environ)

print(flux.job.submit(f, io_jobreq))

Now use the flux CLI to verify that you have a `compute.py` job running on two nodes and that there is one node running the `io-forwarding.py` job.

In [ ]:
!flux jobs

To see how many of your allocation's resources are being consumed by these jobs use the `flux resource list` command.

In [ ]:
!flux resource list

Next, launch another `compute.py` and `io-forwarding.py` job pair that spans all the nodes in your allocation.

In [ ]:
nt = int(nn*2)
cpt = int((nc-nn)/nt)
         
compute_jobreq = JobspecV1.from_command(
    command=["./flux-workflow-examples/job-submit-api/compute.py", "120"], num_tasks=nt, num_nodes=nn, cores_per_task=cpt
)
compute_jobreq.cwd = os.getcwd()
compute_jobreq.environment = dict(os.environ)

print("compute.py job:", flux.job.submit(f, compute_jobreq))

io_jobreq = JobspecV1.from_command(
    command=["./flux-workflow-examples/job-submit-api/io-forwarding.py", "120"], num_tasks=nn, num_nodes=nn, cores_per_task=1
)
io_jobreq.cwd = os.getcwd()
io_jobreq.environment = dict(os.environ)

print("io-forwarding.py job:", flux.job.submit(f, io_jobreq))

Unless you waited for the previous jobs to finish, you should see that the new `compute.py` and `io-forwarding.py` jobs are in the _suspended_ state, indicated by an **S** in the `ST` column of the `flux jobs` output. They are suspended because your allocation didn't have the resources, _i.e._, nodes and cores, free to run them.

In [ ]:
!flux jobs

When the first pair of jobs complete, the resources they were using will be freed up and the next pair will entering _running_ state.